In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

def one_hot_encode(data, columns):
    # Kategorik sütunları seçme
    categorical_data = data[columns]

    # One-Hot Encoder oluşturma
    encoder = OneHotEncoder(sparse=False, drop='first')

    # Kategorik veriyi dönüştürme
    encoded_data = encoder.fit_transform(categorical_data)

    # One-Hot Encoding sonucunu DataFrame'e dönüştürme
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns))

    # Dönüştürülmüş veriyi orijinal veriyle birleştirme
    preprocessed_data = pd.concat([data.drop(columns, axis=1), encoded_df], axis=1)

    return preprocessed_data




def min_max_scale(data, column):
    # Min-Max Scaler oluşturma
    scaler = MinMaxScaler()

    # Sütunu seçme ve ölçeklendirme
    scaled_column = scaler.fit_transform(data[[column]])

    # Ölçeklendirilmiş sütunu yeni bir DataFrame'e dönüştürme
    scaled_data = pd.DataFrame(scaled_column, columns=[column])

    # Ölçeklendirilmiş sütunu orijinal veri setine ekleme
    data[column] = scaled_data[column]

    return data


def fill_nan_with_mean(data):
    for column in data.columns:
        if data[column].isnull().any():
            mean = data[column].mean()
            data[column].fillna(mean, inplace=True)
    return data


def veriOnIsleme(data,colomnsForOneHotEncoder,colomnsForMinMaxScaler):
  dataForOneHot=one_hot_encode(data,colomnsForOneHotEncoder)
  for column in colomnsForMinMaxScaler:
    dataForMinMaxScaler = min_max_scale(dataForOneHot, column)
  result=fill_nan_with_mean(dataForMinMaxScaler)
  return result

veriSetim=pd.read_csv("/content/Churn_Modelling.csv")
veriOnİslemeliSet=veriOnIsleme(veriSetim,["Surname","Geography","Gender"],["CreditScore","Age","Tenure","Balance","NumOfProducts","EstimatedSalary"])
veriOnİslemeliSet=veriOnİslemeliSet.drop(["RowNumber","CustomerId"],axis=1)

MyX=veriOnİslemeliSet.drop(["Exited"],axis=1)
MyY=veriOnİslemeliSet["Exited"]

def parcalamaIslemi(A,B):
  from sklearn.model_selection import train_test_split
  X_train, X_test, Y_train, Y_test = train_test_split(A, B,test_size=0.2,random_state=42)
  return X_train, X_test, Y_train, Y_test

MyX_train, MyX_test, MyY_train, MyY_test=parcalamaIslemi(MyX,MyY)

import tensorflow as tf

# BUİLDİNG A NEURAL NETWORK

# set random seed
tf.random.set_seed(42)

# add layer

model=tf.keras.Sequential([
    tf.keras.layers.Dense(150),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(1)
]

)

#compiling

model.compile(loss=tf.keras.losses.mae,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=["mae"]
              )

# fitting

history_model=model.fit(MyX_train,MyY_train,epochs=20)

# evaluating
model.evaluate(MyX_test,MyY_test)

#prediction
import matplotlib.pyplot as plt
y_pred=model.predict(MyX_test)
print(y_pred)
print(MyY_test)
#Plot the history(LOSS GRAPH)
pd.DataFrame(history_model.history).plot()
plt.ylabel("loss")
plt.xlabel("epochs")

pd.DataFrame(history_model.history).plot()
plt.title("model_iris Loss Curve")

veriSetim.corr()['Exited'].sort_values()